# Transformer

----

## Objetivo

Este notebook tiene como finalidad implementar y explicar un modelo Transformer básico utilizando PyTorch, desde una perspectiva educativa y conceptual.

A diferencia del análisis comparativo entre RNN y LSTM (donde se evaluó directamente el desempeño en una tarea de clasificación de sentimiento), aquí el enfoque es comprender los **componentes clave** de la arquitectura Transformer, tales como:

- El mecanismo de atención
- Las capas encoder
- La codificación posicional
- El entrenamiento en paralelo

## Motivación

Aunque el Transformer se ha consolidado como el estándar en tareas avanzadas de NLP (como BERT, GPT, T5, etc.), construir uno desde cero permite entender **cómo y por qué** esta arquitectura supera a las redes recurrentes en muchos contextos.

> ⚠️ Nota: Este notebook no busca superar a RNN o LSTM en métricas de clasificación, sino ilustrar los fundamentos de una arquitectura moderna y liviana de Transformer aplicada a texto.

## Herramientas utilizadas
- PyTorch para la definición del modelo
- NLTK para tokenización
- Dataset de Sentiment140 (mismo que en los otros modelos)

---

## Librerías

In [ ]:
import torch
import torch.nn as nn
import numpy as np

#BLEU
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

In [ ]:
nltk.download('punkt_tab')

## Arquitectura Transformer: Componentes Clave


### Introducción

El modelo Transformer, introducido por Vaswani et al. en 2017, es una arquitectura de deep learning que ha revolucionado el procesamiento de lenguaje natural (NLP). A diferencia de las redes recurrentes (RNN), los Transformers permiten el procesamiento paralelo y capturan dependencias a largo plazo de manera eficiente gracias al mecanismo de atención.

### Encoder-Decoder

El modelo Transformer sigue una arquitectura de **Encoder-Decoder**, donde:

- **Encoder**: Procesa la secuencia de entrada (`input sentence`) y genera una representación contextualizada.
- **Decoder**: Toma la representación del encoder y genera la secuencia de salida (`target sentence`), paso a paso.

Cada uno se compone de múltiples capas idénticas (stacked).

---

## Positional Encoding

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]


### Positional Encoding en el Transformer

En los modelos `Transformer`, a diferencia de los modelos RNN o LSTM, no tiene alguna forma o mecanismo natural para procesar las secuencias en orden temporal paso a paso. Esto se debe a que la atención (attention) opera sobre toda la secuencia al mismo tiempo y por lo tanto no sabe quién va primero y después.

Para poder solucionar esto, se incorpora explícitamente un `Positional Encoding`, el cual añade información de la posición relativa o absoluta de cada token dentro de la secuencia.

Esta codificación de posición se suima a los embeddings de las palabras, de forma que el modelo pueda inferir el orden en el que aparecen las palabras.

En el código implementado, el Positional Encoding funciona de la siguiente manera:

  * Se crea un vector de posiciones (0, 1, 2, 3 ...).

  * Para cada posición, se aplica una combinación de funciones seno y coseno con diferentes frecuencias.

  * Estas funciones trigonométricas generan un patrón de valores que se repite y facilita que el modelo aprenda relaciones de dependencia entre tokens cercanos o lejanos.

  * El resultado es una matriz del mismo tamaño que los embeddings, que se suma elemento a elemento.

  * Así, cada embedding no solo contiene la representación semántica de la palabra, sino también su posición en la frase.

Esto permite al modelo:

  * Diferenciar el primer token del último.

  * Aprender relaciones de largo plazo.

  * Aprovechar la atención de forma más eficiente.

**Motivo de usar funciones seno/coseno**

  * Estas funciones periódicas permiten representar posiciones de manera continua e interpolable.

  * Son invariantes a traslaciones y ayudan a generalizar a secuencias de longitud diferente.

  * Además, no requieren ser aprendidas (a diferencia de un embedding de posición entrenable), reduciendo parámetros y mejorando estabilidad.

---

## Transformer Language Model

In [ ]:
class TransformerTextGenerator(nn.Module):
    def __init__(self, vocab_size, d_model=256, nhead=8, num_layers=4, dim_feedforward=512, max_len=100):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_len)
        self.transformer = nn.Transformer(
            d_model=d_model,
            nhead=nhead,
            num_encoder_layers=num_layers,
            num_decoder_layers=num_layers,
            dim_feedforward=dim_feedforward,
            dropout=0.1,
            batch_first=True
        )
        self.output_layer = nn.Linear(d_model, vocab_size)

    def forward(self, src, tgt, src_mask=None, tgt_mask=None, src_padding_mask=None, tgt_padding_mask=None):
        src = self.embedding(src)
        tgt = self.embedding(tgt)
        src = self.positional_encoding(src)
        tgt = self.positional_encoding(tgt)
        output = self.transformer(
            src=src, tgt=tgt,
            src_mask=src_mask, tgt_mask=tgt_mask,
            src_key_padding_mask=src_padding_mask,
            tgt_key_padding_mask=tgt_padding_mask
        )
        return self.output_layer(output)

### Arquitectura TransformerLM para generación de texto



La clase `TransformerLM` implementa un modelo de lenguaje basado en el bloque Transformer Encoder, diseñado para generación de texto de forma secuencial.
Los componentes del código funcionan de la siguiente forma:

  * **Embedding**  
  Se encarga de transformar cada token (representado por un índice entero) en un vector de dimensión `d_model`. De este modo, cada palabra obtiene una representación semántica densa y continua que puede ser procesada por la red.

  * **Positional Encoding**  
  Como el Transformer no procesa la secuencia en orden explícito, se incorpora una codificación de posición, utilizando funciones seno/coseno, para darle al modelo una noción del orden de las palabras dentro de la frase.

  * **Transformer Encoder**  
  Utiliza múltiples capas (`num_layers`) de bloques de encoder compuestos de:

    * Multi-head self-attention

    * Feedforward

    * Normalización y residual connections

  Este mecanismo le permite capturar dependencias de largo plazo y modelar relaciones entre palabras de manera no secuencial.  
  La atención multi-cabeza facilita que el modelo combine diferentes perspectivas de contexto, con varias “cabezas” especializadas en distintas relaciones dentro de la frase.

- **Máscara causal (src_mask)**  
  Durante la generación de texto, se necesita evitar que el modelo “vea” el futuro. Para ello se aplica una máscara triangular que bloquea posiciones futuras, forzando a predecir la siguiente palabra solo a partir del contexto anterior. Esto es equivalente al comportamiento autoregresivo de un modelo de lenguaje clásico.

- **Capa de salida (Linear)**  
  Finalmente, se proyecta la salida del Transformer Encoder hacia el tamaño del vocabulario, generando para cada posición una distribución de probabilidad sobre el siguiente token posible.

- **Shape management**  
  El modelo transpone los tensores (usando `transpose`) para cumplir con el formato requerido por `nn.TransformerEncoder`, donde la dimensión de la secuencia va primero. Al final se vuelve a transponer para dejar el resultado en el formato tradicional de lotes (batch_size, seq_len, vocab_size).

**Motivo de la arquitectura**  
La arquitectura `TransformerLM` permite generar texto palabra por palabra (next-word prediction), capturando contextos complejos y dependencias de largo plazo. Esto ofrece ventajas notables respecto a arquitecturas RNN tradicionales, como:

  * mayor paralelización

  * mejor capacidad de aprender relaciones distantes
  
  * mayor eficiencia computacional en secuencias largas

En resumen, `TransformerLM` combina embeddings, codificación de posición, atención multi-cabeza y feedforward, constituyendo un modelo de lenguaje potente y flexible para tareas de NLP como la generación de texto.

---

## Generación de texto con máscara causal

In [ ]:
def generate_square_subsequent_mask(sz):
    mask = torch.triu(torch.ones(sz, sz) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

### Implementación de la máscara causal para atención autoregresiva



En el modelo Transformer para generación de texto autoregresiva, es necesario garantizar que el modelo **no pueda ver palabras futuras** al momento de predecir la siguiente palabra. Esto equivale a forzar que cada posición solo tenga acceso a los tokens anteriores, tal como haría un modelo de lenguaje tradicional (por ejemplo un RNN).

La función `generate_square_subsequent_mask(sz)` crea precisamente esta máscara triangular superior, también llamada **máscara causal**, para restringir la atención a los tokens pasados o actuales. Así se evita el *data leakage* durante el entrenamiento.

**Explicación técnica:**
- `torch.ones(sz, sz)` crea una matriz de unos  
- `torch.triu(...).transpose(0,1)` genera la triangular superior transpuesta, dejando unos en las posiciones permitidas  
- Luego se convierte a flotante y se rellena con `-inf` en las posiciones no permitidas, para que el softmax de la atención las ignore  
- En las posiciones permitidas, se pone un 0 para no alterar la puntuación de atención  
- Finalmente, la máscara se retorna con forma `(sz, sz)`

**En resumen**, esta máscara causal impide que el modelo “espíe” tokens futuros, cumpliendo el principio autoregresivo fundamental en modelos de lenguaje basados en Transformer.

---


## Ejemplo de entrenamiento

In [ ]:
def generate_text(model, src, vocab, inv_vocab, max_len=10, bos_token=1, eos_token=2):
    model.eval()
    generated = torch.tensor([[bos_token]], dtype=torch.long)
    with torch.no_grad():
        for _ in range(max_len):
            tgt_mask = generate_square_subsequent_mask(generated.size(1)).to(src.device)
            out = model(src, generated, tgt_mask=tgt_mask)
            next_token = out[:, -1, :].argmax(-1).unsqueeze(1)
            generated = torch.cat((generated, next_token), dim=1)
            if next_token.item() == eos_token:
                break
    return " ".join(inv_vocab[idx.item()] for idx in generated[0][1:] if idx.item() in inv_vocab)

# Vocabulario simple
vocab = {"<pad>":0, "<bos>":1, "<eos>":2, "I":3, "like":4, "cats":5, "dogs":6}
inv_vocab = {idx:tok for tok,idx in vocab.items()}
vocab_size = len(vocab)

# Ejemplo de entrenamiento mínimo
src = torch.tensor([[3, 4, 0]], dtype=torch.long)  # "I like"
tgt_input = torch.tensor([[1, 5]], dtype=torch.long)  # "<bos> cats"
tgt_output = torch.tensor([[5, 2]], dtype=torch.long)  # "cats <eos>"

model = TransformerTextGenerator(vocab_size)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss(ignore_index=0)
tgt_mask = generate_square_subsequent_mask(tgt_input.size(1))

# Entrenamiento simple
for epoch in range(10):
    model.train()
    optimizer.zero_grad()
    output = model(src, tgt_input, tgt_mask=tgt_mask)
    loss = criterion(output.view(-1, vocab_size), tgt_output.view(-1))
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

In [ ]:
torch.save(model.state_dict(), 'transformer_text_generator.pth')

## Generar texto con el modelo entrenado

In [ ]:
# Generar texto
generated = generate_text(model, src, vocab, inv_vocab, max_len=5)
print("Generated:", generated)

In [ ]:
reference = "i like dogs"
candidate = "i love dogs"

In [ ]:
reference_tokens = nltk.word_tokenize(reference)
candidate_tokens = nltk.word_tokenize(candidate)

In [ ]:
# BLEU espera una lista de referencias (en plural)
bleu_score = sentence_bleu(
    [reference_tokens],   # referencias (en plural)
    candidate_tokens,     # hipótesis
    smoothing_function=SmoothingFunction().method1  # suavizado
)

print(f"BLEU score: {bleu_score:.4f}")

## Análisis de resultados

El modelo generó secuencias sintácticamente válidas como:

> **Prompt:** I like  
> **Output:** I like dogs cats like dogs

Si bien repite palabras, respeta la estructura gramatical esperada y utiliza tokens del vocabulario.

El **BLEU score fue 0.1351**, indicando coincidencia parcial con la referencia. Este valor es razonable considerando:

- Vocabulario muy pequeño
- Solo 1 oración de entrenamiento
- Ausencia de embeddings preentrenados
- Pocas épocas de entrenamiento

## Posibles mejoras:

- Usar un dataset real (como Wikitext o un corpus de diálogos)
- Aumentar vocabulario y número de oraciones
- Aumentar num_layers y d_model
- Aplicar *beam search* en la generación en vez de greedy decoding

---

## Conclusión

Este ejercicio permitió entender de forma práctica la estructura de un Transformer básico, resaltando su eficiencia, paralelismo y diseño modular. Para tareas de producción, es recomendable utilizar variantes preentrenadas como BERT, RoBERTa o DistilBERT.

---
